# Learning Pytorch
This series would be taken from various tutorial available in youtube.

## 1 : Creating a simple convolutional neural network
This one is taken from excellent channel Alladin Perrson : https://www.youtube.com/watch?v=Jy4wM2X21u0&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=3

![](https://i.morioh.com/200620/5b0ea047.jpg)

## Handling imports

In [ ]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

## creating the model

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, model_size, num_of_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(model_size, 48)
        self.fc2 = nn.Linear(48, num_of_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, in_channels=1, classes=10):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8, 
            padding=(1,1), 
            kernel_size=(3,3), 
            stride=(1,1)) # same convolution, input will be same as output
        
        self.conv2 = nn.Conv2d(
            in_channels=8, 
            out_channels=16, 
            padding=(1,1), 
            kernel_size=(3,3), 
            stride=(1,1))
        
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.fc1 = nn.Linear(16*7*7, classes)
    
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.pool(out)
        out = F.relu(self.conv2(out))
        out = self.pool(out)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        return out
        
        

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

## Getting device information

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Putting in the hyper parameters

In [ ]:
num_epochs = 5
learning_rate = 0.001
batch_size=64
# input_size=784
in_channels=1
num_of_classes=10


## Getting the dataset

In [ ]:
# try:
#     train_dataset = datasets.MNIST(root='./dataset/', download=True, train=True, transform=transforms.ToTensor())
#     test_dataset = datasets.MNIST(root='./dataset/', download=True, train=False, transform=transforms.ToTensor())
#     train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#     test_dataloader = DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle=True)
# except Exception as e:
#     print(e)
    

since getting a from interenet isn't reliable lets try with a dataset from csv

## Custom data set class

with help from https://www.youtube.com/watch?v=PXOzkkB5eH0

In [ ]:
import numpy as np

testing

In [ ]:
# xy = np.loadtxt('../input/mnist-in-csv/mnist_train.csv', delimiter=",", dtype=np.float32, skiprows=1)
# x = xy[:,1:]
# y = xy[:,[0]]
# print(x, y)
    

In [ ]:
import pandas as pd

dd = pd.read_csv('../input/mnist-in-csv/mnist_train.csv', dtype=np.float)
dd.head()

In [ ]:
y_dash = torch.from_numpy(dd.iloc[:,0].values) # y
y_dash

In [ ]:
type(y_dash)

In [ ]:
dd.shape

In [ ]:
x_dash = torch.from_numpy(dd.iloc[:,1:].values)

# x_dash[0]

In [ ]:
type(x_dash)

In [ ]:
x_d = x_dash.reshape(x_dash.size(0), 1, 28,28)
x_d.shape

testing over

In [ ]:
import numpy as np

class MnistDataset(Dataset):
    def __init__(self, data_path):
        # data loading
        df = pd.read_csv(data_path, dtype=np.float)
        self.x = torch.from_numpy(df.iloc[:,1:].values)
        
        self.x = self.x.reshape(self.x.size(0),1,28,28)
        self.x = self.x.float() # why float?
        
        self.y = torch.from_numpy(df.iloc[:,0].values) 
        self.y = self.y.long() # why long?
        
        self.n_samples = df.shape[0]
        
    def __len__(self):
        return self.n_samples
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

In [ ]:
train_dataset = MnistDataset("../input/mnist-in-csv/mnist_train.csv")
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size,shuffle=True)

test_dataset = MnistDataset("../input/mnist-in-csv/mnist_test.csv")
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size,shuffle=True)


We can also look at individual dataset that loaded

In [ ]:
first = train_dataset[0]
features, label = first
# print(features, label)

In [ ]:
type(features)

## initialising the model

In [ ]:
model = SimpleCNN().to(device=device) # to device?

In [ ]:
#testing
x = torch.randn(64,1,28,28).to(device=device)
y = model(x)
y.shape

## Define loss criterion and optimizer

In [ ]:
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Train on data

In [ ]:
current_loss = 0
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_dataloader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
#         data = data.reshape((data.shape[0],1,28,28))
#         print(f"data shape: {data.shape}")

        # forward
        scores = model(data)
        loss = loss_criterion(scores, targets)
        current_loss = loss
        
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    
    print(f"At epoch: {epoch}, loss: {current_loss}")

In [ ]:
def check_accuracy(model, dataloader):
    
    total_sample = 0
    correct_sample = 0
    
    model.eval()
    
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device=device)
            y = y.to(device=device)
            
#             x = x.reshape(x.shape[0], -1)
            
            scores = model(x)
            _, predictions = scores.max(1)
            correct_sample += (y==predictions).sum()
            total_sample += predictions.size(0)
            
    model.train()
    
    print(f"out of total sample : {total_sample}  correct sample : {correct_sample} accuracy : {float(correct_sample/total_sample)*100:.2f}")
            
    

In [ ]:
check_accuracy(model, train_dataloader)
check_accuracy(model, test_dataloader)